In [1]:
#Importing the necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import rgb2hex
%matplotlib inline
import seaborn as sns

C:\Users\lenovo\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('E:///listings_details.csv')
df.head()

C:\Users\lenovo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (87) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2818,https://www.airbnb.com/rooms/2818,20181206172549,2018-12-06,Quiet Garden View Room & Super Fast WiFi,Quiet Garden View Room & Super Fast WiFi,I'm renting a bedroom (room overlooking the ga...,Quiet Garden View Room & Super Fast WiFi I'm r...,none,"Indische Buurt (""Indies Neighborhood"") is a ne...",...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.10
1,3209,https://www.airbnb.com/rooms/3209,20181206172549,2018-12-06,"Quiet apt near center, great view",You will love our spacious (90 m2) bright apar...,"Our apartment has lots of light, a balcony and...",You will love our spacious (90 m2) bright apar...,none,Welcome to the Spaarndammerbuurt! From the beg...,...,f,NaN,{Amsterdam},f,f,moderate,f,f,1,1.03
2,20168,https://www.airbnb.com/rooms/20168,20181206172549,2018-12-06,100%Centre-Studio 1 Private Floor/Bathroom,"Cozy studio on your own private floor, 100% in...",For those who like all facets of city life. In...,"Cozy studio on your own private floor, 100% in...",none,Located just in between famous central canals....,...,f,NaN,{Amsterdam},f,f,strict_14_with_grace_period,f,f,2,2.18
3,25428,https://www.airbnb.com/rooms/25428,20181206172549,2018-12-06,Lovely apt in City Centre (Jordaan),NaN,"This nicely furnished, newly renovated apt is...","This nicely furnished, newly renovated apt is...",none,NaN,...,f,NaN,{Amsterdam},f,f,strict_14_with_grace_period,f,f,2,0.09
4,27886,https://www.airbnb.com/rooms/27886,20181206172549,2018-12-06,"Romantic, stylish B&B houseboat in canal district",Stylish and romantic houseboat on fantastic hi...,For a romantic couple: A beautifully restored ...,Stylish and romantic houseboat on fantastic hi...,none,"Central, quiet, safe, clean and beautiful.",...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.03


In [3]:
#dropping columns which contain PII and URLs which are not very useful as we are not performing text analysis
df.drop(['id','listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'experiences_offered', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction' , 'house_rules' ,'thumbnail_url' ,'medium_url','picture_url','xl_picture_url', 'host_id', 'host_url', 'review_scores_checkin', 'reviews_per_month','host_name', 'host_location', 'host_since','host_neighbourhood','neighbourhood','jurisdiction_names','state','market','zipcode','state'],axis=1,inplace = True)
df.drop(['host_about', 'host_thumbnail_url','host_picture_url','host_verifications','amenities','calendar_updated','calendar_last_scraped','first_review','host_acceptance_rate','last_review', 'neighbourhood_group_cleansed','square_feet','license', 'weekly_price','monthly_price','security_deposit','cleaning_fee','calculated_host_listings_count'],axis=1,inplace = True)

In [4]:
#dropping records where the values are NULL
df.dropna(subset=['host_response_time','review_scores_location','city'],inplace = True)

In [5]:
df.isna().sum()

host_response_time                  0
host_response_rate                  0
host_is_superhost                   0
host_listings_count                 0
host_total_listings_count           0
host_has_profile_pic                0
host_identity_verified              0
street                              0
neighbourhood_cleansed              0
city                                0
smart_location                      0
country_code                        0
country                             0
latitude                            0
longitude                           0
is_location_exact                   0
property_type                       0
room_type                           0
accommodates                        0
bathrooms                           5
bedrooms                            3
beds                                3
bed_type                            0
price                               0
guests_included                     0
extra_people                        0
minimum_nigh

In [6]:
df['host_response_time'].value_counts()

within an hour        4621
within a few hours    2437
within a day          2184
a few days or more     274
Name: host_response_time, dtype: int64

In [7]:
df['host_response_rate'] = df['host_response_rate'].str.replace('%','',regex=True).astype('float32')
df['price'] = df['price'].str.replace('$','',regex=True)
df['price'] = df['price'].str.replace(',','',regex=True).astype('float32')
df['street'] = df['street'].str.replace('Amsterdam,','',regex=True)
df['street'] = df['street'].str.replace(', Netherlands','',regex=True)

In [8]:

def ind(val):
    if val == 't':
        return 'True'
    elif val == 'f':
        return 'False'
df['host_is_superhost'] = df.apply(lambda x: ind(x.host_is_superhost), axis=1)
df['host_has_profile_pic'] = df.apply(lambda x: ind(x.host_has_profile_pic), axis=1)
df['host_identity_verified'] = df.apply(lambda x: ind(x.host_identity_verified), axis=1)
df['has_availability'] = df.apply(lambda x: ind(x.has_availability), axis=1)
df['host_has_profile_pic'] = df.apply(lambda x: ind(x.host_has_profile_pic), axis=1)
df['host_identity_verified'] = df.apply(lambda x: ind(x.host_identity_verified), axis=1)
df['requires_license'] = df.apply(lambda x: ind(x.requires_license), axis=1)
df['instant_bookable'] = df.apply(lambda x: ind(x.instant_bookable), axis=1)
df['is_business_travel_ready'] = df.apply(lambda x: ind(x.is_business_travel_ready), axis=1)
df['require_guest_profile_picture'] = df.apply(lambda x: ind(x.require_guest_profile_picture), axis=1)
df['require_guest_phone_verification'] = df.apply(lambda x: ind(x.require_guest_phone_verification), axis=1)

In [9]:
# filling the bathrooms, bedrooms and beds with the median values
df['bathrooms'].fillna(df['bathrooms'].median(),inplace=True)
df['bedrooms'].fillna(df['bedrooms'].median(),inplace=True)
df['beds'].fillna(df['beds'].median(),inplace=True)

In [10]:
df.describe()

,host_response_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,...,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_value
count,9516.000000,9516.000000,9516.000000,9516.000000,9516.000000,9516.000000,9516.000000,9516.000000,9516.000000,9516.000000,...,9516.000000,9516.000000,9516.000000,9516.000000,9516.000000,9516.000000,9516.000000,9516.000000,9516.000000,9516.000000
mean,93.994118,7.798340,7.798340,52.366598,4.887254,2.899538,1.142234,1.415826,1.861181,156.556122,...,16.283417,25.853720,85.002942,32.542350,95.074296,9.699769,9.508197,9.796343,9.491803,9.169714
std,16.664925,32.862384,32.862384,0.014438,0.030047,1.336813,1.085288,0.878994,1.427219,120.822784,...,18.473803,29.342725,113.595789,54.342001,6.198066,0.624701,0.778073,0.581490,0.670084,0.755774
min,0.000000,0.000000,0.000000,52.289274,4.763264,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000
25%,100.000000,1.000000,1.000000,52.356899,4.865501,2.000000,1.000000,1.000000,1.000000,99.000000,...,0.000000,0.000000,2.000000,6.000000,93.000000,10.000000,9.000000,10.000000,9.000000,9.000000
50%,100.000000,1.000000,1.000000,52.366017,4.886337,2.000000,1.000000,1.000000,1.000000,130.000000,...,8.000000,12.000000,24.000000,15.000000,97.000000,10.000000,10.000000,10.000000,10.000000,9.000000
75%,100.000000,2.000000,2.000000,52.375435,4.904097,4.000000,1.000000,2.000000,2.000000,180.000000,...,30.000000,51.000000,135.250000,34.000000,99.000000,10.000000,10.000000,10.000000,10.000000,10.000000
max,100.000000,698.000000,698.000000,52.424641,5.010515,17.000000,100.500000,12.000000,32.000000,5040.000000,...,60.000000,90.000000,365.000000,695.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [11]:
df.duplicated().sum()

0

In [12]:
df['city'].value_counts()

Amsterdam                       9446
Amsterdam-Zuidoost                32
Amsterdam Zuid-Oost                5
Watergraafsmeer                    5
Jordaan                            4
De Pijp                            4
Diemen                             3
Amsterdam (Zunderdorp)             3
Amsterdam                          2
Zeeburg                            2
Amstelveen                         2
Zeeburgerkade                      1
Halfweg                            1
Амстердам                          1
1015 SN Amsterdam                  1
Amsterdam, Noord-Holland, NL       1
Badhoevedorp                       1
Amsterdam Zuidoost                 1
Buitenveldert                      1
Name: city, dtype: int64

In [13]:
df['room_type'].value_counts()

Entire home/apt    7261
Private room       2222
Shared room          33
Name: room_type, dtype: int64

In [14]:
df.columns

Index(['host_response_time', 'host_response_rate', 'host_is_superhost',
       'host_listings_count', 'host_total_listings_count',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood_cleansed', 'city', 'smart_location', 'country_code',
       'country', 'latitude', 'longitude', 'is_location_exact',
       'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'bed_type', 'price', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'requires_license', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_

In [16]:
df['city'].value_counts()

Amsterdam                       9446
Amsterdam-Zuidoost                32
Amsterdam Zuid-Oost                5
Watergraafsmeer                    5
Jordaan                            4
De Pijp                            4
Diemen                             3
Amsterdam (Zunderdorp)             3
Amsterdam                          2
Zeeburg                            2
Amstelveen                         2
Zeeburgerkade                      1
Halfweg                            1
Амстердам                          1
1015 SN Amsterdam                  1
Amsterdam, Noord-Holland, NL       1
Badhoevedorp                       1
Amsterdam Zuidoost                 1
Buitenveldert                      1
Name: city, dtype: int64

In [17]:
df.to_csv('E:\\newone.csv')